llm lacks specific info, cannot answer queries it was not trained on, it hallucinates and give generic responses

RAG is a technique taht combines the capabilities of a pretrained llm with an external datasource

it does this by storing data in external database usually vector dbs or nosql dbs.

so now the llm has the context along with query that helps in the responses

In [2]:
!pip install -q newsapi-python
!pip install -q sentence-transformers
!pip install -q faiss-cpu
!pip install -q langchain
!pip install -q huggingface_hub
!pip install -q transformers
!pip install -q accelerate
!pip install -q bitsandbytes
!pip install -q einops


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 60.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 77.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 51.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 MB 9.9 MB/s eta 0:00:00


In [3]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [4]:
from newsapi import NewsApiClient
import json
import os

# Replace with your actual News API key
api_key = 'c5befea1b83347f488850b449e013abb'
newsapi = NewsApiClient(api_key=api_key)

# Folder path in your Google Drive
folder_path = "/content/drive/MyDrive/news_json/"
os.makedirs(folder_path, exist_ok=True)

# Topics to fetch
topics = ["technology", "finance", "health", "sports", "science"]

for topic in topics:
    articles = newsapi.get_everything(q=topic, language='en', page_size=20)
    with open(f"{folder_path}/{topic}.json", "w") as f:
        json.dump(articles['articles'], f)


In [7]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import os
import json

# Define your news folder path
folder_path = "/content/drive/MyDrive/news_json"

# Load the embedding model
model = SentenceTransformer('all-MiniLM-L6-v2')  # lightweight and fast

documents = []
file_names = os.listdir(folder_path)

for file in file_names:
    file_path = os.path.join(folder_path, file)

    # Skip non-json files
    if not file.endswith(".json"):
        continue

    try:
        with open(file_path, "r", encoding="utf-8") as f:
            articles = json.load(f)

            # Handle top-level dicts
            if isinstance(articles, dict):
                articles = [articles]

            for article in articles:
                # Ensure it's a dictionary with "content"
                if isinstance(article, dict) and article.get("content"):
                    documents.append(article["content"])

    except (UnicodeDecodeError, json.JSONDecodeError, TypeError) as e:
        print(f"Skipping file {file}: {e}")

print(f"Total documents loaded: {len(documents)}")

# Generate embeddings
embeddings = model.encode(documents, show_progress_bar=True)

# Convert to FAISS index
dimension = embeddings[0].shape[0]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(embeddings))

# Save the FAISS index
faiss.write_index(index, os.path.join(folder_path, "faiss_index.index"))

# Save corresponding documents for retrieval later
with open(os.path.join(folder_path, "docs.json"), "w", encoding="utf-8") as f:
    json.dump(documents, f, ensure_ascii=False)

print("✅ FAISS index and documents saved successfully.")


Total documents loaded: 93


Batches:   0%|          | 0/3 [00:00<?, ?it/s]

✅ FAISS index and documents saved successfully.


In [8]:
!pip install huggingface_hub[hf_xet]


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.5/53.5 MB 16.5 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import json

# Load the FAISS index and documents
index = faiss.read_index("/content/drive/MyDrive/news_json/faiss_index.index")

with open("/content/drive/MyDrive/news_json/docs.json", "r") as f:
    documents = json.load(f)

# Load same embedding model
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# User query
query = "What’s the latest on global health?"

query_vector = embedding_model.encode([query])
D, I = index.search(np.array(query_vector), k=5)

# Top 5 results
context = "\n".join([documents[i] for i in I[0]])

# Load Zephyr or similar model for language translation / generation
model_id = "HuggingFaceH4/zephyr-7b-alpha"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype="auto"
)

rag_prompt = f"""
You are an intelligent assistant. Based on the news context below, respond concisely in Tamil.

News Context:
{context}

Answer in Tamil:
"""

inputs = tokenizer(rag_prompt, return_tensors="pt")
inputs = {k: v.to("cpu") for k, v in inputs.items()}

outputs = model.generate(**inputs, max_new_tokens=256, do_sample=True, temperature=0.7)
response = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(response)


tokenizer_config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

...

model-00004-of-00008.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

model-00008-of-00008.safetensors:   0%|          | 0.00/816M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [ ]:
prompt = "Translate the following sentence to Tamil: 'The government released a new policy today.'"
inputs = tokenizer(prompt, return_tensors="pt").to("cpu")
outputs = model.generate(**inputs, max_new_tokens=100)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


In [ ]:
!pip install -q sentence-transformers faiss-cpu google-generativeai googletrans==4.0.0-rc1


In [ ]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import os
import json

model = SentenceTransformer('all-MiniLM-L6-v2')  # Lightweight sentence transformer
folder_path = "/content/drive/MyDrive/news_json"

documents = []
file_names = os.listdir(folder_path)

for file in file_names:
    file_path = os.path.join(folder_path, file)

    if not file.endswith(".json"):
        continue

    try:
        with open(file_path, "r", encoding="utf-8") as f:
            articles = json.load(f)
            for article in articles:
                if isinstance(article, dict) and article.get("content"):
                    documents.append(article["content"])
    except (UnicodeDecodeError, json.JSONDecodeError, AttributeError) as e:
        print(f"Skipping file {file}: {e}")

# Generate embeddings
embeddings = model.encode(documents, show_progress_bar=True)

# Convert to FAISS index
dimension = embeddings[0].shape[0]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(embeddings))

# Save index and documents
faiss.write_index(index, folder_path + "/faiss_index.index")
with open(folder_path + "/docs.json", "w") as f:
    json.dump(documents, f)


In [ ]:
def search_faiss(query, k=3):
    query_embedding = model.encode([query])
    D, I = index.search(np.array(query_embedding), k)
    results = [documents[i] for i in I[0]]
    return results


In [ ]:
import google.generativeai as genai

genai.configure(api_key="YOUR_GEMINI_API_KEY")

def ask_gemini(prompt):
    model = genai.GenerativeModel('gemini-pro')
    chat = model.start_chat()
    response = chat.send_message(prompt)
    return response.text


In [ ]:
from googletrans import Translator

translator = Translator()

def translate(text, dest_language="hi"):  # "hi" = Hindi, "ta" = Tamil, etc.
    translation = translator.translate(text, dest=dest_language)
    return translation.text


In [ ]:
def rag_pipeline(user_query, target_language="hi"):
    print("🔍 Searching documents...")
    top_docs = search_faiss(user_query)

    context = "\n".join(top_docs)
    full_prompt = f"""Answer the following question based on the context below:

    Context:
    {context}

    Question: {user_query}
    Answer:"""

    print("🤖 Generating answer...")
    answer = ask_gemini(full_prompt)

    print("🌐 Translating answer...")
    translated_answer = translate(answer, dest_language=target_language)

    return translated_answer


In [ ]:
result = rag_pipeline("What happened in the Indian stock market last week?", "ta")
print("🗣️ Final Answer in Tamil:\n", result)
